In [ ]:
import pomegranate as pg
import pymc3 as pm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import theano.tensor as T

In [ ]:
signal = np.concatenate([
   np.full((50,), 1.0), 
   np.full((50,), 2.0), 
   np.full((50,), 3.0)
])

growth = np.abs(np.gradient(signal))
growth = growth / np.sum(growth) #normalize

In [ ]:
signal = np.concatenate([
    np.sin(np.linspace(0, 100*np.pi, 1000)),
    np.sin(np.linspace(100*np.pi, 200*np.pi, 1000)) + 10,
    np.sin(np.linspace(100*np.pi, 200*np.pi, 1000)) + 3,
])

growth = np.abs(np.gradient(signal))
growth = growth / np.sum(growth) #normalize
plt.plot(signal)

In [ ]:
plt.plot(growth)

In [ ]:
N = np.arange(0, len(growth))
with pm.Model() as model:
    alpha = 1.0/signal.mean()
    
    lambda_1 = pm.Exponential("lambda_1", alpha)
    lambda_2 = pm.Exponential("lambda_2", alpha)
    lambda_3 = pm.Exponential("lambda_3", alpha)

    tau1 = pm.DiscreteUniform("tau1", lower=N.min(), upper=N.max()) 
    tau2 = pm.DiscreteUniform("tau2", lower=tau1, upper=N.max()) 

    _mu = T.switch(tau1>=N,lambda_1,lambda_2)
    mu = T.switch(tau2>=N,_mu,lambda_3)

    observation = pm.Poisson("obs", mu, observed=signal)
    trace = pm.sample(10000, tune=1000)

In [ ]:
model.vars

In [ ]:
pm.plot_trace(trace);

In [ ]:
pm.summary(trace)

In [ ]:
expected_vals = np.zeros((2, len(signal)))


lambda_1_samples = trace['lambda_1']
lambda_2_samples = trace['lambda_2']
lambda_3_samples = trace['lambda_3']


tau_samples_1 = trace['tau1']
tau_samples_2 = trace['tau2']

N = tau_samples_1.shape[0]

for day in range(0, len(signal)):
    # ix is a bool index of all tau samples corresponding to
    # the switchpoint occurring prior to value of 'day'
    ix1 = day < tau_samples_1
    ix2 = day < tau_samples_2
    # Each posterior sample corresponds to a value for tau.
    # for each day, that value of tau indicates whether we're "before"
    # (in the lambda1 "regime") or
    #  "after" (in the lambda2 "regime") the switchpoint.
    # by taking the posterior sample of lambda1/2 accordingly, we can average
    # over all samples to get an expected value for lambda on that day.
    # As explained, the "message count" random variable is Poisson distributed,
    # and therefore lambda (the poisson parameter) is the expected value of
    # "message count".
    expected_vals[0][day] = (lambda_1_samples[ix1].sum()
                                   + lambda_2_samples[~ix1].sum()) / N
    
    expected_vals[1][day] = (lambda_2_samples[ix2].sum()
                                   + lambda_3_samples[~ix2].sum()) / N
    
expected_vals = np.median(expected_vals, axis=0)

In [ ]:
plt.plot(range(len(signal)), signal)
plt.plot(range(len(signal)), expected_vals)
